In [1]:
%%HTML
<style>.container{width:100%;}</style>

# Tests

In der Datei `test_OrderedSet.py` sind die Tests von `set` und `frozenset` in [CPython](https://github.com/python/cpython/blob/3.7/Lib/test/test_set.py "R. D. Hettinger et al. (2019): cpython/Lib/test/test_set.py, GitHub"). Wir speichern die Tests in einer Datei statt in einem Block, weil wir sie so leichter für die Versionen mit und ohne Cython ausführen können. In dieser Datei wurden einige Details modifiziert:

- `set` und `frozenset` wurden mit IDE-Werkzeugen zu `OrderedSet` und `OrderedFrozenset` umbenannt.
- Notation mit erweiterten regulären Ausdrücken: `{([^:]+?)}` wurde mit `OrderedSet([$1])` ersetzt, sodass Mengenliterale in `OrderedSet`s übersetzt werden, aber keine Dictionary-Literale.
- In `TestBasicOps.check_repr_against_values` ist die Überprüfung der Stringrepräsentation auf die von `OrderedSet` angepasst.
- In `TestJointOps.test_sub_and_super`, `TestSubsets` und `TestIdentities.test_binopsVsSubsets` ist die Verwendung von `<` und `>` auf `is_proper_subset` und `is_proper_superset` angepasst.
- Am Ende des Programms wurde `if __name == '__main__':↵    unittest.main()` zu `unittest.main(argv=[""], exit=False)` geändert, weil die Tests in Jupyter anders als in einem Skript aufgerufen werden müssen.

Einige Tests werden auch übersprungen:

- `TestJointOps.test_iterator_pickling` *pickle*t einen Iterator. Dafür müssten wir explizit ein zustandsbehaftetes Iteratorobjekt schreiben. Dies ist möglich, wird jedoch unterlassen, da die Performancenachteile der Erstellung solcher Objekte zu hoch sind.
- `TestJointOps.test_do_not_rehash_dict_keys` überprüft, ob die Hashwerte bei der Kopie von einem `dict` in ein `set` stabil bleiben. Dies gewährleisten wir nicht, da wir überhaupt keine Hashmaps benutzen, und somit auch keine Hashwerte vorhalten.
- `TestFrozenSet.test_singleton_empty_frozenset` überprüft, dass die Speicheradresse der leeren unveränderlichen Menge immer gleich ist. Dies können wir ohne die Verwendung von C und damit explizitem Speichermanagement nicht abbilden.
- `TestFrozenSet.test_constructor_identity` überprüft, dass die Speicheradressen von zwei soeben erstellten nichtleeren unveränderlichen Mengen mit gleichem Inhalt gleich sind. Auch hierfür wäre manuelles Speichermanagement erforderlich.
- `TestFrozenSet.test_copy` überprüft, dass die Speicheradressen einer Kopie einer unveränderlichen Mengen gleich sind. Wieder wäre manuelles Speichermanagement erforderlich.
- `TestFrozenSet.test_hash_effectiveness` überprüft die Streuung der Hashwerte von verschiedenen unveränderlichen Mengen. Dieser Test basiert auf der Annahme, dass ein Hashalgorithmus verwendet wird, der komplexer ist als der, den wir benutzen.
- Die letzte Zeile von `TestBasicOps.test_iteration` ist auskommentiert, da sie den `__length_hint__` des Iteratorobjektes benutzt. Ein Abbilden dieses Verhaltens würde ein Iteratorobjekt statt eine `__iter__`-Implementierung mit `yield` erfordern (vgl. Kommentar zu `test_iterator_pickling`).

Bei einigen Tests ist im Moment noch unklar, ob sie später übersprungen werden:

- `TestJointOps.test_cyclical_repr` und `TestJointOps.test_cyclical_print` setzen voraus, dass bei der String-Repräsentation von Objekten Zyklen erkannt und mit `...` gekennzeichnet werden.
- `TestExceptionPropagation.test_changingSizeWhileIterating` stellt sicher, dass die Modifikation einer Menge, während über sie iteriert wird, verboten ist.

Dies gilt auch für einige Tests, die nur im Falle der Verwendung von Cython übersprungen werden.

- `TestJointOps.check_free_after_iterating` gibt mit Cython `ValueError`s. Die verwendete Funktion ist nur für die CPython-interne Entwicklung und nicht weiter dokumentiert:

In [2]:
import test.support
help(test.support.check_free_after_iterating)

Help on function check_free_after_iterating in module test.support:

check_free_after_iterating(test, iter, cls, args=())



- Der Teil von `TestJointOps.test_pickling`, in dem überprüft wird, ob Attribute, die Objekten von Unterklassen der Mengen hinzugefügt werden, beim Pickling beibehalten werden, wird übersprungen, da diese Funktionalität nicht mit Cythons Klassenstrukturen kompatibel scheint.
- `TestFrozenSet.test_init` überprüft, ob, wenn auf ein bestehendes `OrderedFrozenset` nochmal `__init__` aufgerufen wird, die Menge wirklich eingefroren bleibt. Diese Funktionalität bilden wir derzeit nicht ab (vgl. `Cython.ipynb`).
- `TestFrozenSet.test_hash`, `TestFrozenSet.test_frozen_as_dictkey` und `TestFrozenSet.test_hash_caching` geben mit Cython bei manchen (nicht bei allen) Durchläufen `OverflowError`s, dass der Hashwert einer unveränderlichen eingefrorenen Menge zu groß sei.

Wir testen zunächst die Version ohne Cython.

In [3]:
%run 2-Ordered-Sets.ipynb

Help on class set in module builtins:

class set(object)
 |  set() -> new empty set object
 |  set(iterable) -> new set object
 |  
 |  Build an unordered collection of unique elements.
 |  
 |  Methods defined here:
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __contains__(...)
 |      x.__contains__(y) <==> y in x.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iand__(self, value, /)
 |      Return self&=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __isub__(self, value, /)
 |      Return self-=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __ixor__(self, value, /)
 |      Re

Wir schalten Linting aus, insbesondere, weil einige Zeilen nun länger sind als im Original, vor allem wegen der Änderungen zu `OrderedSet` bzw. `OrderedFrozenset`. Auch ist das Linting nicht vollständig mit den Magic-Kommandos kompatibel. Die Tests, die wir bei Cython überspringen, überspringen wir nicht, dies ist mit der Variable `cython_skippings` gekennzeichnet. Die Tests lassen wir mit `%run -i` laufen, so werden sie im laufenden Namespace dieses Notebooks ausgeführt.

In [4]:
%flake8_off
cython_skippings = False
%run -i test_OrderedSet.py

................................................................................................................................................................................................s....s..sss..s......s...s.....s.............ss..s......s...s........................................................................................................................s.....ss....s............s............................s.....ss....s............s...........................s.....ss....s............s........................................................
----------------------------------------------------------------------
Ran 561 tests in 3.294s

OK (skipped=29)


Wir testen auch mit dem Notebook `3-Cython.ipynb`. Diesmal überspringen wir die mit unserer Cython-Implementierung inkompatiblen Tests.

In [5]:
%run 3-Cython.ipynb
cython_skippings = True
%run -i test_OrderedSet.py

................................................................................................................................................................................................s....s..sss..s.ss.ssss..s.....s.............ss..s.ss.ssss..s........................................................................................................................s.....ss....s.s..........s............................s.....ss....s.s..........s...........................s.....ss....s.s..........s........................................................
----------------------------------------------------------------------
Ran 561 tests in 1.953s

OK (skipped=42)
